In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import Policy, Records, Calculator, Behavior, behavior
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.transforms import BlendedGenericTransform
%matplotlib inline
import copy

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_reform = Policy()
records_reform = Records(puf)

calcbase = Calculator(policy = policy_base, records = records_base)
calcreform = Calculator(policy = policy_reform, records = records_reform)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform = {
          2015:{
        "_ID_InterestPaid_HC":[0.5],
        "_ID_StateLocalTax_HC":[0.5],
        "_ID_Charity_HC":[0.5]
    }
}
policy_reform.implement_reform(reform)

In [ ]:
calcbase.advance_to_year(2015)
calcreform.advance_to_year(2015)

In [ ]:
calcbase.calc_all()
calcreform.calc_all()

In [ ]:
def agg_diff(calcX, calcY):
    df_x = results(calcX)
    df_y = results(calcY)
    
    agg_x = sum(df_x['_combined']*df_x['s006'])
    agg_y = sum(df_y['_combined']*df_y['s006'])
    
    agg_diif = agg_y - agg_x

    agg_diif.tofile('agg_diif.csv',sep=',',format='%1.3f')

In [ ]:
agg_diff(calcbase,calcreform)

In [ ]:
def agg_num_delta(calcX, calcY):
    df_x = results(calcX)
    df_y = results(calcY)
    
    id_itemizers_x = ((calcX.records.c04470 > 0) & (calcX.records.c00100 > 0))
    id_itemizers_y = ((calcY.records.c04470 > 0) & (calcY.records.c00100 > 0))
    df_x['pct_itm'] = id_itemizers_x
    df_y['pct_itm'] = id_itemizers_y
    
    itm_x = sum(df_x['pct_itm'] * df_x['s006'])
    itm_y = sum(df_y['pct_itm'] * df_y['s006'])
    
    agg_num_delta = itm_y - itm_x

    agg_num_delta.tofile('agg_num_delta.csv',sep=',',format='%1.3f')

In [ ]:
agg_num_delta(calcbase,calcreform)

In [ ]:
def func(reform_on_II):
    policy_func = Policy()
    records_func = Records(puf)
    calc_func = Calculator(policy = policy_func, records = records_func)
    reform = {
        2015:{
        "_II_rt1":[max(0.1 + reform_on_II,0)],
        "_II_rt2":[max(0.15 + reform_on_II,0)],
        "_II_rt3":[max(0.25 + reform_on_II,0)],
        "_II_rt4":[max(0.28 + reform_on_II,0)],
        "_II_rt5":[max(0.33 + reform_on_II,0)],
        "_II_rt6":[max(0.35 + reform_on_II,0)],
        "_II_rt7":[max(0.396+ reform_on_II,0)]}
    }
    policy_func.implement_reform(reform)
    calc_func.advance_to_year(2015)
    calc_func.calc_all()
    return (calc_func.records._combined*calc_func.records.s006).sum()


In [ ]:
def bisec(calcY, epsilon):
    target = (calcY.records._combined*calcY.records.s006).sum()
    upp = 0.1
    low = -0.1
    mid = (upp + low)/2.0
    while (upp - low)/2.0 > epsilon:
        rvn = func(mid)
        if rvn == target:
            return mid 
        elif rvn < target:
            low = mid
        else:
            upp = mid
        mid = (upp + low)/2.0
    return mid

In [ ]:
bisec(calcreform, 0.0001)